# The fundarmental group of $\Sigma_{g,1}$ 

In [1]:
require './freegroup.rb'

true

## generators

In [2]:
genus = 3
alphabet = %w(a b c d e f)

Gens = [Letter.new] + alphabet.map{|x| Letter.new(x)}
Gens.freeze
p Gens[1..-1].map(&:show)

a = [Gens[0]] + Gens[1..-1].each_slice(2).map(&:first)
b = [Gens[0]] + Gens[1..-1].each_slice(2).map(&:last)
p a.map(&:show)
nil

["a", "b", "c", "d", "e", "f"]
["1", "a", "c", "e"]


In [3]:
comms = []
(Gens.size/2 + 1).times do |i|
  comms << Group.commutator(a[i], b[i])
end
p comms.map(&:show)

["1111", "abAB", "cdCD", "efEF"]


["1111", "abAB", "cdCD", "efEF"]

In [4]:
Rseq = []
(Gens.size/2).times do |i|
  Rseq += [a[i+1], b[i+1].inverse, a[i+1].inverse, b[i+1]]
end
Rseq.freeze
p Rseq.map(&:show) #.index('A')
nil

<main>:2: warning: already initialized constant Rseq
<main>:2: warning: already initialized constant Rseq
<main>:2: warning: previous definition of Rseq was here
<main>:2: warning: already initialized constant Rseq
<main>:2: warning: previous definition of Rseq was here


["a", "B", "A", "b", "c", "D", "C", "d", "e", "F", "E", "f"]


## Turaev cobraket

In [5]:
def ep(h1, h2)
  result = h1[:sign]*h2[:sign]
  result *= if h1[:letter].show == h2[:letter].show
    (h1[:index] < h2[:index]) ? h2[:sign] : h1[:sign]*(-1)
  else
    (Rseq.index(h1[:letter]) < Rseq.index(h2[:letter])) ? 1 : -1
  end
  return result
end
#--------------------------------------------

def lk(word, div_pair)
  word.flatten!
  div_hashs = div_pair.map do |k|
    [{index: k%(word.size)+1, letter: word[k%(word.size)], sign: 1},
       {index: k, letter: word[k-1].inverse, sign: -1}]
  end
  sum = 0
  2.times do |i|
    2.times{|j| sum += ep(div_hashs[0][i], div_hashs[1][j])}
  end
  return (-1)*(sum/2)
end; nil

In [6]:
def cobracket(myw, verbose: false)
  
  #-----
  # 2つの div の組み合わせ全てについて、{divs: *, linking: *, words: *} という Hash を生成。
  
  divs = (1..myw.size).to_a
  monomials = divs.combination(2).map do |combi|
    #--- dividng word --- 
    facs = myw.flatten.factors
    words = Array.new(2)
    words[0] = Word.new(facs[0..combi[0]-1]+facs[combi[1]..-1])
    words[1] = Word.new(facs[combi[0]..combi[1]-1])
    
    {divs: combi, linking: lk(myw, combi), words: words}
  end
  #-----
  
  if verbose
    result = monomials
  else
    monomials.delete_if{|mono| mono[:linking] == 0}
    monomials.each{|mono| mono[:words] = mono[:words].map(&:cyclic_reduce)}
    result = []
    while monomials.size > 0
      pop = monomials.pop
      pop[:divs] = [pop[:divs]]
      pop[:coeff] = pop[:linking]
      #---
      monomials.each do |mono|
        if mono[:words][0].conjugate?(pop[:words][0]) and mono[:words][1].conjugate?(pop[:words][1])
          pop[:coeff] += mono[:linking]
          pop[:divs] << mono[:divs]
          mono[:linking] = nil
        elsif mono[:words][0].conjugate?(pop[:words][1]) and mono[:words][1].conjugate?(pop[:words][0])
          pop[:coeff] += mono[:linking]*(-1)
          pop[:divs] << mono[:divs]
          mono[:linking] = nil
        end
      end
      #---
      if pop[:coeff] != 0 or verbose
        result << pop
      end
      #---
      monomials.delete_if{|mono| mono[:linking].nil?}
    end 
  end
  return (result.empty?) ? [{divs: [], coeff: 0, words: []}] : result
end; nil

In [7]:
mw = Word.new(b[1], a[1], b[1]) #a1*Group.commutator(b[1], a[2]) #comms[1] #
printf mw.show + "\n-------\n"

vbs = true #false #
cobracket(mw, verbose: vbs).each do |h|
  h.each do |k,v| 
    v = v.map(&:show).join(" \u{2227} ") if k == :words
    p "#{k}: #{v}"
  end
  printf "---\n"
end; nil

bab
-------
"divs: [1, 2]"
"linking: 0"
"words: bb ∧ a"
---
"divs: [1, 3]"
"linking: 1"
"words: b ∧ ab"
---
"divs: [2, 3]"
"linking: 1"
"words: ba ∧ b"
---


# Execution

In [25]:
a1, b1 = Word.new(a[1]), Word.new(b[1])
c = (a1*Word.new(b[1])*(a1.inverse)).flatten

samples = {
  s0: [
    a1, 
    a1*Word.new(b[2]),
    a1*(Word.new(b[2]).inverse),
    a1*c*(c.inverse),
  ],
  s1: [
    a1^4,
    c,
    comms[1],
    Word.new(b[1])*a1*Word.new(b[1])
  ],
  s2: [
    a1*(comms[2].inverse),
    a1*(c.inverse)*(comms[2].inverse)*c,
    a1*(comms[2].inverse)*Group.commutator(comms[2], c.inverse),
    (a1*(comms[2].inverse)*Group.commutator(comms[2], c.inverse)).cyclic_reduce
  ],
  s3: [
    a1*comms[2],
    a1*(comms[2]^2),
    a1*comms[2]*c*comms[2]*(c.inverse),
    a1*(comms[2]^2)*Group.commutator(comms[2].inverse,c)
  ],
  s4: [
    a1*(Word.new(a[2])^3),
    a1*(Word.new(a[2], b[2].inverse)^3)
  ],
  s5: [
    a1*Group.commutator(a[2], a[3]),
    a1*(comms[2]*comms[3])*Group.commutator(a[2], a[3])*((comms[2]*comms[3]).inverse)
  ],
  s6: [
    a1*Group.commutator(b[1], a[2]),
    a1*(a1.inverse)*Word.new(b[1])*a1,
    c*(a1.inverse)*a1,
    c*(c.inverse),
    Word.new(a[1], b[1], b[1].inverse, b[2])
  ],
  s7: [
    a1*(Group.conjugate(a1, b1.inverse))*a1*(b1.inverse),
    a1*b1*((a1*(b1.inverse))^2),
    a1*Group.conjugate(b1.inverse, a1)*a1*(b1.inverse),
    (a1*Group.conjugate(b1.inverse, a1)*a1*(b1.inverse)).contract,
    a1*a1*(Word.new(b[2].inverse)^2)
    ]
  }

arr = samples.slice(:s7) #:s0, :s6) #:s2) #, :s3) #:s0, :s1) #
arr.each do |k, v|
  printf "--- #{k} ---\n "
  v.each do |s|
    ws = s.show
    cob = cobracket(s, verbose: false).map do |h|
     coeff = (h[:coeff]==1) ? '' : "(#{h[:coeff].to_s})"
#       coeff = (h[:linking]==1) ? '' : "(#{h[:linking].to_s})"
      coeff + "#{h[:words].join(" \u{2227} ")}"
    end
    #---
    puts "#{ws}  |--\u{03B4}-->  #{cob.join(' + ')}"
  end
end;nil

--- s7 ---
 a.Bab.a.B  |--δ-->  (0)
a.b.a.B.a.B  |--δ-->  (0)
a.aBA.a.B  |--δ-->  aB ∧ aB
aaBB  |--δ-->  (-1)aB ∧ aB
a.a.D.D  |--δ-->  aaD ∧ D + aDD ∧ a


## Experiments

In [9]:
#myw = Word.new(a[2], a[1].inverse, b[2])*Word.new(a[1], a[1].inverse)*Word.new(b[1], a[2]) #.contract
myw = Word.new(a[2].inverse)*Word.new(a[1])*Word.new(a[2], a[2].inverse)
p myw.show

vb = true #false #
printf("=====\n")
cobracket(myw, verbose: vb).each do |h|
  h.each do |k,v|
    case k 
    when :divs, :linking
      p "#{k}: #{v}"
    when :factors
      p h[:factors].map(&:show).join(" , ")
    when :words
      coeff = (h[:coeff]==1) ? '' : "(#{h[:coeff].to_s})"
#      p coeff + "#{h[:words].map(&:cyclic_reduce).join(" \u{2227} ")}"
      p "words: " + "#{h[:words].map(&:cyclic_reduce).join(" \u{2227} ")}"
    else
    end
  end
  printf("---\n")
end; nil

"C.a.cC"
=====
"divs: [1, 2]"
"linking: 1"
"words: C ∧ a"
---
"divs: [1, 3]"
"linking: 0"
"words: CC ∧ ac"
---
"divs: [1, 4]"
"linking: 1"
"words: C ∧ a"
---
"divs: [2, 3]"
"linking: 0"
"words: CaC ∧ c"
---
"divs: [2, 4]"
"linking: -1"
"words: Ca ∧ 1"
---
"divs: [3, 4]"
"linking: 1"
"words: a ∧ C"
---


In [10]:
myw = samples[:s6][-1]*Word.new(a[1].inverse, b[2].inverse)
p myw.show

vb = false #true #
printf("=====\n")
cobracket(myw, verbose: vb).each do |h|
  h.each do |k,v|
    case k 
    when :divs, :linking
      p "#{k}: #{v}"
    when :factors
      p h[:factors].map(&:show).join(" , ")
    when :words
      coeff = (h[:coeff]==1) ? '' : "(#{h[:coeff].to_s})"
#      p coeff + "#{h[:words].map(&:cyclic_reduce).join(" \u{2227} ")}"
      p "words: " + "#{h[:words].map(&:cyclic_reduce).join(" \u{2227} ")}"
    else
    end
  end
  printf("---\n")
end; nil

"abBd.AD"
=====
"divs: [[5, 6], [1, 4], [3, 4]]"
"linking: 1"
"words: d ∧ D"
---
"divs: [[3, 6]]"
"linking: -1"
"words: a ∧ A"
---
"divs: [[3, 5]]"
"linking: 1"
"words: aD ∧ dA"
---
"divs: [[1, 3]]"
"linking: -1"
"words: adAD ∧ 1"
---


### TODO

* [ ] cobracket を再帰的に作用させて、完全に分解することに何らかの意味があるか?
* [ ] $\ell_{2}$ の計算と cobracket との関係は? とくに両者にある division について。
* [x] a.abA.aBA の cobraket にバグあり。
* [x] div 同士の linking を div に対応する2文字ワード同士の単純な計算で求められないか?  <-- **DONE!**
* [x] 係数がプラマイ逆かな。
* [x] simple subword たちによる計算を実装する。<-- ダメだった。各 simple subword が内部では確実に交差しない保証がないとダメだから。